In [69]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
# from mlfs import config
# settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /Users/fredrikstrom/Documents/KTH_Dokument/Scalable ML/labs/mlfs-book


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [70]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [72]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-13 23:31:28,861 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-13 23:31:28,863 INFO: Initializing external client
2025-11-13 23:31:28,864 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-13 23:31:30,491 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1286301


'{"country": "sweden", "city": "stockholm", "street": "st-eriksgatan-83", "aqicn_url": "https://api.waqi.info/feed/@10523", "latitude": 59.33, "longitude": 18.07}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [73]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=3,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

In [74]:
air_quality_fg_2 = fs.get_feature_group(
    name='air_quality',
    version=1,
)

In [75]:
air_quality_fg_2.read().sort_values(by=['date']).tail(5)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.10s) 


,date,pm25,country,city,street,url
401,2025-11-06 00:00:00+00:00,29.0,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
2224,2025-11-07 00:00:00+00:00,39.0,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
2373,2025-11-08 00:00:00+00:00,64.0,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
2515,2025-11-11 00:00:00+00:00,21.0,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523
2516,2025-11-12 00:00:00+00:00,23.0,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523


---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


## Grade C task 

In [76]:
aq_lag_features = air_quality_fg.read()
aq_lag_features = aq_lag_features.sort_values(by=['date'])

last_3 = aq_lag_features.tail(3)
last_3 

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.40s) 


,date,pm25,country,city,street,url,pm25_lag_1d,pm25_lag_2d,pm25_lag_3d
664,2025-11-11 00:00:00+00:00,21.0,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523,64.0,39.0,29.0
2229,2025-11-12 00:00:00+00:00,23.0,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523,21.0,64.0,39.0
2517,2025-11-13 00:00:00+00:00,6.0,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523,23.0,21.0,64.0


In [77]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,12.0,sweden,stockholm,st-eriksgatan-83,2025-11-13,https://api.waqi.info/feed/@10523


In [78]:
# Ensure both are timezone-aware UTC
last_3["date"] = pd.to_datetime(last_3["date"], utc=True)
aq_today_df["date"] = pd.to_datetime(aq_today_df["date"], utc=True)

combined_aq_df = pd.concat([last_3, aq_today_df], ignore_index=True).sort_values(by=["date"])

combined_aq_df = combined_aq_df.drop(
    columns=[c for c in combined_aq_df.columns if c.startswith("pm25_lag_")],
    errors="ignore",
)

combined_aq_df["pm25_lag_1d"] = combined_aq_df["pm25"].shift(1).astype("Float64")
combined_aq_df["pm25_lag_2d"] = combined_aq_df["pm25"].shift(2).astype("Float64")
combined_aq_df["pm25_lag_3d"] = combined_aq_df["pm25"].shift(3).astype("Float64")

aq_today_with_lags = combined_aq_df.iloc[-1:].copy()

In [79]:
aq_today_with_lags

,date,pm25,country,city,street,url,pm25_lag_1d,pm25_lag_2d,pm25_lag_3d
3,2025-11-13 00:00:00+00:00,12.0,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523,6.0,23.0,21.0


In [80]:
aq_today_with_lags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 3 to 3
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype              
---  ------       --------------  -----              
 0   date         1 non-null      datetime64[ns, UTC]
 1   pm25         1 non-null      float32            
 2   country      1 non-null      object             
 3   city         1 non-null      object             
 4   street       1 non-null      object             
 5   url          1 non-null      object             
 6   pm25_lag_1d  1 non-null      Float64            
 7   pm25_lag_2d  1 non-null      Float64            
 8   pm25_lag_3d  1 non-null      Float64            
dtypes: Float64(3), datetime64[ns, UTC](1), float32(1), object(4)
memory usage: 203.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [81]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 59.25°N 18.0°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-13,8.65,0.2,16.595179,294.341064,stockholm
1,2025-11-14,3.00,0.0,11.726277,287.878784,stockholm
2,2025-11-15,1.65,0.0,10.799999,270.000000,stockholm
3,2025-11-16,3.10,0.0,11.119281,240.945480,stockholm
4,2025-11-17,0.35,0.0,12.475961,313.830933,stockholm
5,2025-11-18,-0.35,0.0,7.928178,267.397491,stockholm
6,2025-11-19,-0.40,0.3,10.931203,342.758453,stockholm


In [82]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 352.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

,date,pm25,country,city,street,url,pm25_lag_1d,pm25_lag_2d,pm25_lag_3d
3,2025-11-13 00:00:00+00:00,6.0,sweden,stockholm,st-eriksgatan-83,https://api.waqi.info/feed/@10523,23.0,21.0,64.0


In [83]:
# Insert new data
air_quality_fg.insert(combined_aq_df)

2025-11-13 23:32:33,706 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286301/fs/1273931/fg/1668839


Uploading Dataframe: 100.00% |██████████| Rows 4/4 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: air_quality_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286301/jobs/named/air_quality_3_offline_fg_materialization/executions


(Job('air_quality_3_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 739489
         }
       },
       "result": {
         "observed_value": 6.0,
         "element_count": 4,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-13T10:32:33.000706Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectati

In [84]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-13 23:33:21,403 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286301/fs/1273931/fg/1668709


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286301/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-13 23:33:38,914 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-13 23:33:42,130 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-13 23:35:41,196 INFO: Waiting for execution to finish. Current state: SUCCEEDING. Final status: UNDEFINED
2025-11-13 23:35:44,423 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-13 23:35:44,614 INFO: Waiting for log aggregation to finish.
2025-11-13 23:36:00,071 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 739415
         }
       },
       "result": {
         "observed_value": 7.928177833557129,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-13T10:33:21.000402Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": 

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
